## Keras Tuner

L'utilisation de KerasTuner va nous permettre de trouver facilement les hyperparamètres optimaux pour avoir de meilleurs résultats avec notre modèle.


In [7]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kerastuner as kt
import numpy as np
import pathlib

# Générateurs d'images
img_generator = ImageDataGenerator(rescale=1./255)
aug_img_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    brightness_range=[0.5, 1.5],
    zoom_range=.3
)

# Variables globales
data_dir = pathlib.Path('./seg_train')
image_count = len(list(data_dir.glob('*/*.jpg')))

test_dir = pathlib.Path('./seg_test')
test_count = len(list(test_dir.glob('*/*.jpg')))

BATCH_SIZE = 32
IMG_HEIGHT = 150
IMG_WIDTH = 150
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
VALIDATION_STEPS = np.ceil(test_count/BATCH_SIZE)
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*')])
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
VALIDATION_STEPS = np.ceil(test_count/BATCH_SIZE)

# Générateur des données de tests
test_data_gen = img_generator.flow_from_directory(
    directory=str(test_dir),
    batch_size=BATCH_SIZE,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

# Générateur des données d'entrainement, augmentées
aug_train_data_gen = aug_img_generator.flow_from_directory(
    directory=str(data_dir),
    batch_size=BATCH_SIZE,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

def build_model(hp):
    model = keras.Sequential()
    model.add(
        keras.layers.Conv2D(
            filters=hp.Int('conv2d_filters', 16, 128, 16),
            kernel_size=hp.Int('conv2d_kernel', 3, 7, 2),
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
            padding=hp.Choice('padding', ['same', 'valid']),
            activation='relu'
        ),
    )
    model.add(keras.layers.MaxPool2D(pool_size=hp.Int('maxpool2d', 1, 3)))
    
    for i in range(hp.Int('conv2d', 0, 3)):
        model.add(keras.layers.Conv2D(
            filters=hp.Int('conv2d_filters_' + str(i), 16, 128, 16),
            kernel_size=hp.Int('conv2d_kernel_' + str(i), 3, 7, 2),
            activation='relu'
        ))
        model.add(keras.layers.MaxPool2D(pool_size=hp.Int('maxpool2d_' + str(i), 1, 3)))
    
    model.add(keras.layers.Flatten()),
    
    for i in range(hp.Int('dense', 1, 3)):
        model.add(keras.layers.Dense(hp.Int('dense_units_' + str(i), 32, 128, 32), activation='relu'))
        
    model.add(keras.layers.Dropout(hp.Float('dropout', 0., .5)))
    
    model.add(keras.layers.Dense(len(CLASS_NAMES), activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

tuner = kt.tuners.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=5,
    project_name='tuner'
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner.search(
    x=aug_train_data_gen,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=10,
    validation_data=test_data_gen,
    validation_steps=VALIDATION_STEPS,
    callbacks=[callback],
    verbose=2
)

tuner.results_summary()

Found 3000 images belonging to 6 classes.
Found 14034 images belonging to 6 classes.
INFO:tensorflow:Reloading Oracle from existing project .\tuner\oracle.json
INFO:tensorflow:Reloading Tuner from .\tuner\tuner0.json
INFO:tensorflow:Oracle triggered exit
